<a href="https://colab.research.google.com/github/MatthiasSimons/covid-analysis/blob/main/covid_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programmierung und Auswertung großer Datenmengen
## Analyse der Covid-19 Fallzahlen und Impfungen

# Einleitung
...

Vorgehensweise


*   Importieren
*   Bereinigen
*   Modifizieren
*   Modellieren
*   Validieren
*   Bewerten
*   Zusammenfassen

# Gliederung
CRoos Industry Standard Process for Data Mining (CRISP-DM)

1.   Business Understanding
2.   Data Understanding
3.   Data Preperation
4.   Modelling
5.   .... 



## Vorbereitung

Importieren der Bibliotheken



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 1. Business Understanding
... 

# 2. Data Understanding
- zwei Datensätze über die Covid19 Fallzahlen und Impfrate (Link einfügen)

## 2.1 Importieren der Daten

In [59]:
url_fallzahlen = "https://raw.githubusercontent.com/ocaktans/Gathering-Covid-19-Data/73623b5057979a4bca4ffdf931cc29222ad7a58e/Covid-19%20Dataset.csv"
url_impfungen = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv"
fallzahlen = pd.read_csv(url_fallzahlen, sep=',')
fallzahlen.name = "Fallzahlen"
impfungen = pd.read_csv(url_impfungen, sep=',')
impfungen.name = "Impfungen"

## 2.2 Beschreibung der Daten

### Datensatz: Covid-19 Fallzahlen

Bei dem Covid-19-Data Datensatz handelt es sich um einen Datensatz der die Entwicklung der Covid-19 Fallzahlen weltweit täglich abbildet.

In [ ]:
fallzahlen.head()

,Unnamed: 0,Country,Date,Confirmed,Recovered,Deaths,Active Cases,Death Rate(%),Increase,FirstDays,day_xth,Population_1k,TenCase,Cases/1000s
0,0,Afghanistan,2020-01-22,0,0,0,0,0.0,0,2020-02-24,0,38928,2020-03-11,0.0
1,1,Afghanistan,2020-01-23,0,0,0,0,0.0,0,2020-02-24,0,38928,2020-03-11,0.0
2,2,Afghanistan,2020-01-24,0,0,0,0,0.0,0,2020-02-24,0,38928,2020-03-11,0.0
3,3,Afghanistan,2020-01-25,0,0,0,0,0.0,0,2020-02-24,0,38928,2020-03-11,0.0
4,4,Afghanistan,2020-01-26,0,0,0,0,0.0,0,2020-02-24,0,38928,2020-03-11,0.0


Der Datensatz besteht aus 14 Spalten die im folgenden kurz beschrieben werden:
- Country: Land
- Date: Datum
- Confirmed: Kumulative bestätigte Fälle
- Recovered: Kumulative Genesene Fälle
- Deaths: Kumulative Verstorbene
- Active Cases: Aktive Fälle
- Death Rate(%):
- Increase: Tägliche Steigerung der bestätigten Fälle
- FirstDays: Tag an dem der erste Fall gemeldet wurde
- day_xth: verstrichene Tage seit dem ersten gemeldeten Fall
- Population_1k: Einwohnerzahl in Tausend
- TenCase: Tag an dem Zehn Fälle gemeldet wurden
- Cases/1000s: Fälle pro Tausend Einwohner

### Datensatz: Covid-19 Impfrate

Bei diesem Datensatz handelt es sich um einen Datensatz der die Entwicklung der Covid-19 Impfungen weltweit täglich abbildet

In [ ]:
impfungen.head()

,location,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0


Der Datensatz besteht aus 12 Spalten die im folgenden kurz beschrieben werden:
- location: Land
- iso_code: Kodierung des Landes
- date: Datum
- total_vaccinations: Anzahl Impfungen
- people_vaccinated: Geimpfte Personen
- people_fully_vaccinated: Vollständig geimpfte Personen
- daily_vaccinations_raw: Tägliche Impfungen (Roh)
- daily_vaccinations: Tägliche Impfungen
- total_vaccinations_per_hundred: Anzahl Impfungen auf 100 Einwohner
- people_vaccinated_per_hundred: Anzahl geimpfte Personen auf 100 Einwohner
- people_fully_vaccinated_per_hundred: Anzahl vollständig geimpfte Personen auf 100 Einwohner
- daily_vaccinations_per_million: Anzahl Impfungen auf eine Millionen Einwohner

# 3. Data Preperation
Verbesserung der Datenqualität durch:

*   Zusammenfassen zu einem Dataframe
*   Löschen von Zellen
*   Leere Zellen



## 3.1 Zusammenfassen zu einem Datensatz

In [75]:
# geht das?

## 3.2 Daten säubern

*   Prüfung und Umgang mit fehlenden Daten



In [71]:
def get_null_columns(df):
      print("Es fehlen",str(df.isnull().sum().sum()), "Einträge in", str(df.name))
      return [index for index in df.isnull().sum().index if df.isnull().sum()[index] > 0]

fallzahlen_null_columns = get_null_columns(fallzahlen)
impfungen_null_columns = get_null_columns(impfungen)
print("fehlende Spalten in Fallzahlen",fallzahlen_null_columns)
print("fehlende Spalten in Impfungen",impfungen_null_columns)

Es fehlen 2415 Einträge in Fallzahlen
Es fehlen 70842 Einträge in Impfungen
fehlende Spalten in Fallzahlen ['TenCase']
fehlende Spalten in Impfungen ['total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'daily_vaccinations_raw', 'daily_vaccinations', 'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred', 'daily_vaccinations_per_million']


Im Datensatz Fallzahlen fehlen nur in der Spalte "TenCase" Daten. Da die Spalte für die Auswertung nicht weiter relevant ist, wird sie gelöscht.

In [73]:
try:
  fallzahlen.drop(columns="TenCase", inplace = True, axis = 1)
except:
  print("Zelle bereits ausgeführt")

print("Es fehlen ",str(fallzahlen.isnull().sum().sum()), " Einträge in Fallzahlen")

Zelle bereits ausgeführt
Es fehlen  0  Einträge in Fallzahlen


Im Datensatz "Impfungen" fehlen in 9 von 12 Spalten Werte.

In [74]:
impfungen_null_columns

['total_vaccinations',
 'people_vaccinated',
 'people_fully_vaccinated',
 'daily_vaccinations_raw',
 'daily_vaccinations',
 'total_vaccinations_per_hundred',
 'people_vaccinated_per_hundred',
 'people_fully_vaccinated_per_hundred',
 'daily_vaccinations_per_million']

In [16]:
impfungen.isnull().sum()

location                                   0
iso_code                                   0
date                                       0
total_vaccinations                      8586
people_vaccinated                       9333
people_fully_vaccinated                12020
daily_vaccinations_raw                 10512
daily_vaccinations                       226
total_vaccinations_per_hundred          8586
people_vaccinated_per_hundred           9333
people_fully_vaccinated_per_hundred    12020
daily_vaccinations_per_million           226
dtype: int64